In [1]:
__NAME = 'Laplace grif3ex4'
# root_path = '/content/drive/MyDrive/Colab Notebooks/Code/Laplace Eqn'
# %tensorflow_version 1.x
# !pip install pyDOE

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# import os
# os.chdir(root_path)
# !pwd

# Imports

In [4]:
import tensorflow as tf
print(tf.version)
print(tf.test.is_built_with_cuda())
print(tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

<module 'tensorflow._api.v1.version' from 'I:\\Users\\Ragav\\miniconda3\\envs\\juptenflowgpu115\\lib\\site-packages\\tensorflow_core\\_api\\v1\\version\\__init__.py'>
True
True


In [5]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# from plotting import newfig, savefig
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [6]:
import numpy as np
import scipy.io
from scipy.interpolate import griddata
import time
from pyDOE import lhs

In [7]:
import pickle as pkl

In [8]:
%matplotlib widget

# Load Data

# Model

In [9]:
# Layers
u_layers = [2, 50, 50, 50, 50, 1]
pde_layers = [3, 100, 100, 1]

layers = [2, 50, 50, 50, 50, 1]

layers = [2, 10, 50, 100, 100, 50, 30, 30, 1]

In [10]:
# tf placeholders for Identification
t_tf = tf.placeholder(tf.float32, shape=[None, 1])
x_tf = tf.placeholder(tf.float32, shape=[None, 1])
u_tf = tf.placeholder(tf.float32, shape=[None, 1])
t_tf, x_tf, u_tf

(<tf.Tensor 'Placeholder:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'Placeholder_1:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'Placeholder_2:0' shape=(?, 1) dtype=float32>)

In [11]:
def initialize_NN(layers):
    weights = []
    biases = []
    num_layers = len(layers)
    for l in range(0, num_layers - 1):
        W = xavier_init(size=[layers[l], layers[l + 1]])
        b = tf.Variable(tf.zeros([1, layers[l + 1]], dtype=tf.float32),
                        dtype=tf.float32)
        weights.append(W)
        biases.append(b)
    return weights, biases

def xavier_init(size):
    in_dim = size[0]
    out_dim = size[1]
    xavier_stddev = np.sqrt(2 / (in_dim + out_dim))
    return tf.Variable(tf.truncated_normal([in_dim, out_dim],
                                           stddev=xavier_stddev,
                                           dtype=tf.float32),
                       dtype=tf.float32)

In [12]:
def neural_net(X, weights, biases):
    num_layers = len(weights) + 1
    H = X
    for l in range(0, num_layers - 2):
        W = weights[l]
        b = biases[l]
        H = tf.sin(tf.add(tf.matmul(H, W), b))
    W = weights[-1]
    b = biases[-1]
    Y = tf.add(tf.matmul(H, W), b)
    return Y

In [13]:
weights, biases = initialize_NN(layers)
# weights, biases

In [14]:
# load weights and biases

with open(__NAME + '/weights.pkl', 'rb') as db_file:
    W_pkl = pkl.load(db_file)

with open(__NAME + '/biases.pkl', 'rb') as db_file:
    B_pkl = pkl.load(db_file)

W = []
B = []
for w, b in zip(W_pkl, B_pkl):
    W.append(tf.Variable(w))
    B.append(tf.Variable(b))

weights = W
biases = B

In [15]:
lb_tf = tf.placeholder(tf.float32, shape=[2])
ub_tf = tf.placeholder(tf.float32, shape=[2])

In [16]:
# tf placeholders for Solution
xx_lb_tf = tf.placeholder(tf.float32, shape=[None, 1])
yx_lb_tf = tf.placeholder(tf.float32, shape=[None, 1])
ux_lb_tf = tf.placeholder(tf.float32, shape=[None, 1])

xx_ub_tf = tf.placeholder(tf.float32, shape=[None, 1])
yx_ub_tf = tf.placeholder(tf.float32, shape=[None, 1])
ux_ub_tf = tf.placeholder(tf.float32, shape=[None, 1])

xy_lb_tf = tf.placeholder(tf.float32, shape=[None, 1])
yy_lb_tf = tf.placeholder(tf.float32, shape=[None, 1])
uy_lb_tf = tf.placeholder(tf.float32, shape=[None, 1])

xy_ub_tf = tf.placeholder(tf.float32, shape=[None, 1])
yy_ub_tf = tf.placeholder(tf.float32, shape=[None, 1])
uy_ub_tf = tf.placeholder(tf.float32, shape=[None, 1])

x_f_tf = tf.placeholder(tf.float32, shape=[None, 1])
y_f_tf = tf.placeholder(tf.float32, shape=[None, 1])

In [17]:
def sol_net_u(x, y):
    X = tf.concat([x, y], 1)
    H = 2.0 * (X - lb_tf) / (ub_tf - lb_tf) - 1.0
    u = neural_net(H, weights, biases)
    u_x = tf.gradients(u, x)[0]
    u_y = tf.gradients(u, y)[0]
    return u, u_x, u_y

def sol_net_f(x, y):
    u, u_x, u_y = sol_net_u(x, y)

    u_xx = tf.gradients(u_x, x)[0]
    u_yy = tf.gradients(u_y, y)[0]

    f = u_xx + u_yy 

    return f

In [18]:
# tf graphs for Solution
ux_lb_pred, ux_x_lb_pred, ux_y_lb_pred = sol_net_u(xx_lb_tf, yx_lb_tf)
ux_ub_pred, ux_x_ub_pred, ux_y_ub_pred = sol_net_u(xx_ub_tf, yx_ub_tf)
uy_lb_pred, uy_x_lb_pred, uy_y_lb_pred = sol_net_u(xy_lb_tf, yy_lb_tf)
uy_ub_pred, uy_x_ub_pred, uy_y_ub_pred = sol_net_u(xy_ub_tf, yy_ub_tf)
sol_f_pred = sol_net_f(x_f_tf, y_f_tf)

In [19]:
# loss for Solution
sol_loss =  tf.reduce_sum(tf.square(ux_lb_tf - ux_lb_pred)) + \
            tf.reduce_sum(tf.square(ux_ub_tf - ux_ub_pred)) + \
            tf.reduce_sum(tf.square(uy_lb_tf - uy_lb_pred)) + \
            tf.reduce_sum(tf.square(uy_ub_tf - uy_ub_pred)) + \
            tf.reduce_sum(tf.square(sol_f_pred))

In [20]:
# Optimizer for Solution
sol_optimizer = tf.contrib.opt.ScipyOptimizerInterface(
    sol_loss,
    var_list = weights + biases,
    method='L-BFGS-B',
    options={
        'maxiter': 50000,
        'maxfun': 50000,
        'maxcor': 50,
        'maxls': 50,
        'ftol': 1.0 * np.finfo(float).eps
    })

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [21]:
adam_optimizer = tf.train.AdamOptimizer()
sol_train_op_Adam = adam_optimizer.minimize(
            sol_loss,
            var_list= weights + biases)

In [22]:
# tf session
sess = tf.Session(config=tf.ConfigProto(
    allow_soft_placement=True, log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5



# Training

## Prepare data

In [23]:
lb = np.array([-1.0, 0.0])
ub = np.array([ 1.0, 1.0])

In [24]:
N = 10
fig = plt.figure()
ax = fig.gca()
ax.set_xlim(lb[0], ub[0])
ax.set_ylim(lb[1], ub[1])
ax.set_xticks(np.arange(lb[0],ub[0],(ub[0] - lb[0])/N))
ax.set_yticks(np.arange(lb[1],ub[1],(ub[1] - lb[1])/N))
plt.grid()
l = lb + (ub - lb) * lhs(2, N) 
plt.scatter(l[:, 0], l[:, 1], color="r", label="lhs")
plt.title("Latin Hypercube Sampling\nN=10")
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
fig.set_figheight(3.8)
fig.set_figwidth(6)
plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [71]:
Nx = 800
Ny = 800
N_f = 40000
(Nx, Ny, N_f)

(800, 800, 40000)

In [72]:
x_data = np.linspace(lb[0], ub[0], Nx)[:, None]
y_data = np.linspace(lb[1], ub[1], Ny)[:, None]

In [73]:
X_sol, Y_sol = np.meshgrid(x_data, y_data)
# U_sol = u_data

In [74]:
X_sol_star = np.hstack(
    (X_sol.flatten()[:, None],
    Y_sol.flatten()[:, None])
    )
# U_sol_star = U_sol.flatten()[:, None]

print(X_sol_star.shape, X_sol_star[0:10], sep='\n')

(640000, 2)
[[-1.          0.        ]
 [-0.99749687  0.        ]
 [-0.99499374  0.        ]
 [-0.99249061  0.        ]
 [-0.98998748  0.        ]
 [-0.98748436  0.        ]
 [-0.98498123  0.        ]
 [-0.9824781   0.        ]
 [-0.97997497  0.        ]
 [-0.97747184  0.        ]]


In [75]:
a = 1
b = 1
V0 = 1

In [76]:
uy0 = 0 * x_data
uya = 0 * x_data
uxb = V0 * (0 * y_data + 1)
ux_b = V0 * (0 * y_data + 1)

In [77]:
XYx_lb = np.hstack((
    - b + 0 * y_data,
    y_data
 ))                                         # x lower boundary boundary
Ux_lb = ux_b

XYx_ub = np.hstack((
    b + 0 * y_data,
    y_data
 ))                                         # x lower boundary boundary
Ux_ub = uxb

In [78]:
# shuffled initial boundary data (x boundary)
idx_x = np.random.choice(y_data.shape[0], y_data.shape[0], replace=False)
XYx_lb_train = XYx_lb[idx_x, :]
Ux_lb_train = Ux_lb[idx_x, 0:1]
XYx_ub_train = XYx_ub[idx_x, :]
Ux_ub_train = Ux_ub[idx_x, 0:1]

In [79]:
XYy_lb = np.hstack((
    x_data,
    0 * x_data
 ))                                         # x lower boundary boundary
Uy_lb = uy0

XYy_ub = np.hstack((
    x_data,
    a + 0 * x_data
 ))                                         # x lower boundary boundary
Uy_ub = uya

In [80]:
# shuffled initial boundary data (y boundary)
idx_y = np.random.choice(x_data.shape[0], x_data.shape[0], replace=False)
XYy_lb_train = XYy_lb[idx_y, :]
Uy_lb_train = Uy_lb[idx_y, 0:1]
XYy_ub_train = XYy_ub[idx_y, :]
Uy_ub_train = Uy_ub[idx_y, 0:1]

In [81]:
XY_f_train = lb + (ub - lb) * lhs(2, N_f)

In [82]:
XYy_lb_train.max()

1.0

In [83]:
fig, ax = plt.subplots()
ax.set_xlim(lb[0] -0.1, ub[0] + 0.1)
ax.set_ylim(lb[1] - 0.1, ub[1] + 0.1)
fig.set_figheight(6)
fig.set_figwidth(6)

ax.scatter(XYx_lb_train[:, 0], XYx_lb_train[:, 1], s=4, marker='.')
ax.scatter(XYx_ub_train[:, 0], XYx_ub_train[:, 1], s=4, marker='.')
ax.scatter(XYy_lb_train[:, 0], XYy_lb_train[:, 1], s=4, marker='.')
ax.scatter(XYy_ub_train[:, 0], XYy_ub_train[:, 1], s=4, marker='.')

ax.scatter(XY_f_train[:, 0], XY_f_train[:, 1], s=4, marker='.', edgecolors='none')
# ax.imshow(u0_train, extent=(t_data[0, 0], t_data[1, 0], x0_train.max(), x0_train.min()), aspect='auto')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [84]:
def callback(loss):
    print('Loss: %e' % (loss))

In [85]:
tf_dict = {
    lb_tf: lb,
    ub_tf: ub,
    xx_lb_tf: XYx_lb_train[:, 0:1],
    yx_lb_tf: XYx_lb_train[:, 1:2],
    ux_lb_tf: Ux_lb_train,
    xx_ub_tf: XYx_ub_train[:, 0:1],
    yx_ub_tf: XYx_ub_train[:, 1:2],
    ux_ub_tf: Ux_ub_train,
    xy_lb_tf: XYy_lb_train[:, 0:1],
    yy_lb_tf: XYy_lb_train[:, 1:2],
    uy_lb_tf: Uy_lb_train,
    xy_ub_tf: XYy_ub_train[:, 0:1],
    yy_ub_tf: XYy_ub_train[:, 1:2],
    uy_ub_tf: Uy_ub_train,
    x_f_tf: XY_f_train[:, 0:1],
    y_f_tf: XY_f_train[:, 1:2]
}

In [86]:
start_time = time.time()
it = 0
end = True
while not(end):

    sess.run(sol_train_op_Adam, tf_dict)

    # Print
    if it % 10 == 0:
        elapsed = time.time() - start_time
        loss_value = sess.run(sol_loss, tf_dict)
        print('It: %d, Loss: %.3e, Time: %.2f' %
                (it, loss_value, elapsed))
        start_time = time.time()

        if loss_value < 5 * 10**(-1):
            end = True

    it = it + 1

In [87]:
# sol_optimizer.minimize(sess,
#                        feed_dict=tf_dict,
#                        fetches=[sol_loss],
#                        loss_callback=callback)

In [88]:
sess.run(sol_loss, feed_dict=tf_dict)

78.09792

In [89]:
with open(__NAME + '/weights.pkl', 'wb') as db_file:
    pkl.dump(obj=sess.run(weights), file=db_file)

with open(__NAME + '/biases.pkl', 'wb') as db_file:
    pkl.dump(obj=sess.run(biases), file=db_file)

In [90]:
X_sol_star[:, 0:1]

array([[-1.        ],
       [-0.99749687],
       [-0.99499374],
       ...,
       [ 0.99499374],
       [ 0.99749687],
       [ 1.        ]])

In [91]:
u_pred = sess.run(ux_lb_pred, {
        lb_tf: lb,
        ub_tf: ub,
        xx_lb_tf: X_sol_star[:, 0:1],
        yx_lb_tf: X_sol_star[:, 1:2]
    })

In [92]:
fig = plt.figure(figsize=(4*1.75,4), dpi=200)
ax = fig.gca()

ax.set_xlim(lb[0], ub[0])
ax.set_ylim(lb[1], ub[1])

# plt.subplots_adjust(bottom=0.17)
# plt.subplots_adjust(left=0.17)

plt.title('V')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')

plt.pcolormesh(np.reshape(X_sol_star[:, 0], (Nx, -1)), 
               np.reshape(X_sol_star[:, 1], (Nx, -1)), 
               np.reshape(u_pred[:, 0], (Nx, -1)), 
               shading='gouraud',  cmap='jet')
plt.colorbar()

plt.tight_layout()
plt.savefig('Figures\\Laplace 1.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …